In [1]:
import pandas as pd
import numpy as np
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import matplotlib.pyplot as plt
from tqdm import tqdm
import datetime
from scipy.signal import argrelextrema

/home/gs/python/amb/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# Percorso della cartella
folder_path = "./input"
#apri url live
print('Carica URL del sito live, oppure URL in 404, che dovranno venire redirezionati')
# Lista dei file Excel nella cartella
excel_files = [f for f in os.listdir(folder_path) if f.endswith(".xlsx") or f.endswith(".xls")]

# Controlla se ci sono file Excel nella cartella
if excel_files:
    print("Ecco l'elenco dei file Excel presenti nella cartella:")
    for index, file in enumerate(excel_files, start=1):
        print(f"{index}. {file}")
else:
    print("Nessun file Excel trovato nella cartella specificata.")

Carica URL del sito live, oppure URL in 404, che dovranno venire redirezionati
Ecco l'elenco dei file Excel presenti nella cartella:
1. 404_gsc.xlsx
2. live_internal_html.xlsx


In [3]:
# fogli disponibili
while True:
    try:
        file_number = int(input("Carica URL del sito live, oppure URL in 404, che dovranno venire redirezionati. Inserisci il numero corrispondente al file da caricare: "))
        if 1 <= file_number <= len(excel_files):
            filename = os.path.join(folder_path, excel_files[file_number - 1])
            xl = pd.ExcelFile(filename)
            break
        else:
            print("Il numero inserito non corrisponde a nessun file. Riprova.")
    except ValueError:
        print("Si prega di inserire un numero valido.")

# ottieni la lista dei nomi dei fogli del file Excel
nomi_fogli = xl.sheet_names

# stampa i nomi dei fogli
print("I fogli disponibili sono:")
for i, nome in enumerate(nomi_fogli):
    print(f"{i + 1}. {nome}")

I fogli disponibili sono:
1. Table


In [4]:
# chiedi all'utente di selezionare un foglio da leggere
while True:
    try:
        indice_foglio = int(input("Inserisci il numero del foglio con gli URL da redirezionare: ")) - 1
        if 0 <= indice_foglio < len(nomi_fogli):
            nome_foglio = nomi_fogli[indice_foglio]
            break
        else:
            print("Il numero inserito non è valido. Riprova.")
    except ValueError:
        print("Inserisci un numero valido. Riprova.")
    except Exception as e:
        print(f"Si è verificato un errore: {e}. Riprova.")

# leggi il foglio selezionato dall'utente
df = pd.read_excel(filename, sheet_name=nome_foglio)
df.head(5)
print('foglio letto')

# Crea una copia del dataframe originale
df1 = df.copy()

# Mostra all'utente le colonne disponibili nel dataframe
print("Here are the columns available in the dataframe:")
for i, col in enumerate(df1.columns):
    print(f"{i}: {col}")

foglio letto
Here are the columns available in the dataframe:
0: URL


In [5]:
# Chiedi all'utente di selezionare la colonna da mantenere
colonna_index = input("Inserisci il numero colonna con gli URL da redirezionare: ")

# Continua a chiedere all'utente di inserire una colonna finché non viene fornita un'input valida
while not colonna_index.isdigit() or int(colonna_index) >= len(df1.columns):
    colonna_index = input("Invalid input. Please enter a valid column number: ")

colonna_da_mantenere = df1.columns[int(colonna_index)]

In [6]:
# Cancella tutte le colonne tranne quella selezionata dall'utente
df1 = df1[[colonna_da_mantenere]]

# rimuovi le righe con valori nulli o vuoti
df1 = df1.dropna()
df1.rename(columns={colonna_da_mantenere: "LIVE URLS"}, inplace=True)
print('foglio rinominato')
# togli duplicazioni
df1.drop_duplicates(subset='LIVE URLS', keep='first', inplace=True)
print('duplicaizoni rimosse')
# Stampa il dataframe risultante
df1.head(5)

foglio rinominato
duplicaizoni rimosse


,LIVE URLS
0,https://www.nonnapaperina.it/2014/04/la-federc...
1,https://www.nonnapaperina.it/2014/04/limportan...
2,https://www.nonnapaperina.it/2017/10/petto-dan...
3,https://www.nonnapaperina.it/2013/03/fave-con-...
4,https://www.nonnapaperina.it/2016/05/celiachia...


In [7]:
#apri URL dev
if excel_files:
    print("Ecco l'elenco dei file Excel presenti nella cartella. Selezione il file DEV con gli URL finali:")
    for index, file in enumerate(excel_files, start=1):
        print(f"{index}. {file}")

Ecco l'elenco dei file Excel presenti nella cartella. Selezione il file DEV con gli URL finali:
1. 404_gsc.xlsx
2. live_internal_html.xlsx


In [8]:
while True:
    try:
        file_number = int(input("Inserisci il numero corrispondente al file da caricare con gli URL finali: "))
        if 1 <= file_number <= len(excel_files):
            filename = os.path.join(folder_path, excel_files[file_number - 1])
            xl = pd.ExcelFile(filename)
            break
        else:
            print("Il numero inserito non corrisponde a nessun file. Riprova.")
    except ValueError:
        print("Si prega di inserire un numero valido.")

nomi_fogli2 = xl.sheet_names
# stampa i nomi dei fogli
print("I fogli disponibili sono:")
for i, nome in enumerate(nomi_fogli2):
    print(f"{i + 1}. {nome}")
   

I fogli disponibili sono:
1. 1 - HTML


In [9]:
# chiedi all'utente di selezionare un foglio da leggere con gestione caratteri speciali nei nomi dei fogli
while True:
    try:
        indice_foglio2 = int(input("Inserisci il numero del foglio che desideri leggere con gli URL finali: ")) - 1
        if 0 <= indice_foglio2 < len(nomi_fogli2):
            break
        else:
            print("Il numero inserito non è valido. Riprova.")
    except ValueError:
        print("Inserisci un numero valido. Riprova.")
    except Exception as e:
        print(f"Si è verificato un errore: {e}. Riprova.")

# leggi il foglio selezionato dall'utente
df2 = pd.read_excel(filename, sheet_name=indice_foglio2)
df2.head(5)
print('foglio letto')
df2.head(5)


foglio letto


,Address
0,https://www.nonnapaperina.it/
1,https://www.nonnapaperina.it/pillole/
2,https://www.nonnapaperina.it/ingredienti/
3,https://www.nonnapaperina.it/scuola-di-cucina/...
4,https://www.nonnapaperina.it/contattami/


In [10]:
# Mostra all'utente le colonne disponibili nel dataframe
print("Queste sono le colonne disponibili nel foglio:")
for i, col in enumerate(df2.columns):
    print(f"{i}: {col}")

Queste sono le colonne disponibili nel foglio:
0: Address


In [44]:
# Crea una copia del dataframe originale
df22 = df2.copy()

# Chiedi all'utente di selezionare la colonna da mantenere
colonna_index2 = input("inserisci il numero della colonna che contiene gli URL finali: ")

# Continua a chiedere all'utente di inserire una colonna finché non viene fornita un'input valida
while not colonna_index2.isdigit() or int(colonna_index2) >= len(df22.columns):
    colonna_index2 = input("Invalid input. Please enter a valid column number: ")

colonna_da_mantenere2 = df22.columns[int(colonna_index2)]

# Cancella tutte le colonne tranne quella selezionata dall'utente
df22 = df22[[colonna_da_mantenere2]]

# rimuovi le righe con valori nulli o vuoti
df22 = df22.dropna()
df22.rename(columns={colonna_da_mantenere2: "DEV URLS"}, inplace=True)
print('foglio rinominato')
# togli duplicazioni
df22.drop_duplicates(subset='DEV URLS', keep='first', inplace=True)
print('duplicaizoni rimosse')
# Stampa il dataframe risultante
df22.head(5)


foglio rinominato
duplicaizoni rimosse


,DEV URLS
0,https://www.nonnapaperina.it/
1,https://www.nonnapaperina.it/pillole/
2,https://www.nonnapaperina.it/ingredienti/
3,https://www.nonnapaperina.it/scuola-di-cucina/...
4,https://www.nonnapaperina.it/contattami/


In [45]:
old_urls = df1
new_urls = df22

In [46]:
old_urls_backup = old_urls.copy()
old_urls.head(2)

,LIVE URLS,Cleaned LIVE URLS
0,https://www.nonnapaperina.it/2014/04/la-federc...,2014 04 la federcasalinghe e il mondo delle i...
1,https://www.nonnapaperina.it/2014/04/limportan...,2014 04 limportanza delle pentole per la nost...


In [47]:
new_urls_backup = new_urls.copy()
new_urls.head(2)

,DEV URLS
0,https://www.nonnapaperina.it/
1,https://www.nonnapaperina.it/pillole/


In [48]:
# trasforma URL in stringa di testo pulito
# pulizia pre-processing degli URL
from urllib.parse import urlparse, unquote
import re

def clean_url(url):
    # Parse the URL
    parsed = urlparse(unquote(url))
    
    # Remove the scheme and netloc (protocol and domain)
    # parsed.netloc contains the domain, and parsed.scheme contains the protocol (http or https).
    # We are not using these components in the cleaned URL.
    
    # Separate the path of the URL into segments
    # The path is separated into segments based on the '/' character.
    # The path is then rejoined into a single string with the segments separated by ' ' instead of '/'.
    path = ' '.join(parsed.path.split('/'))
    
    # Remove or normalize URL parameters
    # If the URL has parameters, these are normalized by sorting them alphabetically.
    # This ensures that URLs with the same parameters in a different order are considered equal.
    # toglie parametri
    params = parsed.params
    #tiene parametri
    #params = parsed.query
    if params:
        params = '&'.join(sorted(params.split('&')))
    
    # Replace hyphens and underscores with spaces in the path
    path = re.sub(r'[-_#]', ' ', path)
    
    # Return the cleaned URL, which now consists of the path and the normalized parameters.
    # Note that the '?' is only included if there are parameters.
    return path + ('?' + params if params else '')

# Apply the cleaning function to each URL in the dataframes
old_urls['Cleaned LIVE URLS'] = old_urls['LIVE URLS'].apply(clean_url)
new_urls['Cleaned DEV URLS'] = new_urls['DEV URLS'].apply(clean_url)

In [49]:
old_urls.head(2)

,LIVE URLS,Cleaned LIVE URLS
0,https://www.nonnapaperina.it/2014/04/la-federc...,2014 04 la federcasalinghe e il mondo delle i...
1,https://www.nonnapaperina.it/2014/04/limportan...,2014 04 limportanza delle pentole per la nost...


In [50]:
new_urls.head(2)

,DEV URLS,Cleaned DEV URLS
0,https://www.nonnapaperina.it/,
1,https://www.nonnapaperina.it/pillole/,pillole


In [51]:
# calcolo similitudine con fuzz.token_sort_ratio dalla libreria fuzzywuzzy
# calcolo matrice di similarità
import sys

n_print = 100  # Stampa il progresso ogni 100 cicli

similarity_matrix = np.zeros((len(old_urls), len(new_urls)))
print('Calculating similarity between Cleaned URLs')

for i, old_url in enumerate(old_urls['Cleaned LIVE URLS']):
    for j, new_url in enumerate(new_urls['Cleaned DEV URLS']):
        similarity_matrix[i, j] = fuzz.token_sort_ratio(old_url, new_url)

        # Stampa il progresso ogni n_print cicli
        if (i * len(new_urls['Cleaned DEV URLS']) + j) % n_print == 0:
            sys.stdout.write(f'\rProgress: {i * len(new_urls["Cleaned DEV URLS"]) + j}/{len(old_urls["Cleaned LIVE URLS"]) * len(new_urls["Cleaned DEV URLS"])}')
            sys.stdout.flush()

max_similarity_scores = np.max(similarity_matrix, axis=1)

Calculating similarity between Cleaned URLs
Progress: 283000/283024

In [52]:
max_similarity_scores

array([50., 57., 83., 72., 57., 77., 58., 63., 71., 63., 62., 61., 48.,
       70., 54., 95., 78., 64., 72., 72., 61., 89., 65., 76., 66., 78.,
       92., 59., 45., 52., 93., 65., 63., 64., 88., 62., 31., 66., 59.,
       67., 64., 67., 67., 61., 78., 56., 65., 59., 76.])

In [53]:
# cerca di trovare una soglia ottimale utilizzando l'algoritmo 
# del "gomito" (elbow) per l'analisi di un insieme di punteggi di similarità massima.

# opzione nel caso non trovasse thresold
print("Finding optimal threshold...")
hist, bin_edges = np.histogram(max_similarity_scores, bins='auto', density=True)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
hist_derivative = np.diff(hist) / np.diff(bin_centers)
extrema = argrelextrema(hist_derivative, np.greater)

# Check if extrema exist
if extrema[0].size > 0:
    elbow_index = extrema[0][0]
    threshold = bin_centers[elbow_index]
else:
    print("No extrema found. Falling back to default threshold of 50.")
    threshold = 1


Finding optimal threshold...


In [54]:
print(hist_derivative)

[-4.98246173e-04  1.49473852e-03  1.62630326e-18  3.98596939e-03
  1.49473852e-03 -4.48421556e-03  4.98246173e-04 -1.99298469e-03
  9.96492347e-04]


In [55]:
threshold

40.599999999999994

In [56]:
# mapping con FuzzyWuzzy
def find_best_match(old_url, new_urls, threshold=threshold):
    best_score = 0
    best_match = None
    for new_url in new_urls['Cleaned DEV URLS']:
        score = fuzz.token_sort_ratio(old_url, new_url)
        if score > best_score:
            best_score = score
            best_match = new_url

    if best_score >= threshold:
        # Ottieni l'indice della corrispondenza migliore nella colonna 'Cleaned DEV URLS'
        best_match_index = new_urls.index[new_urls['Cleaned DEV URLS'] == best_match].tolist()[0]
        # Ottieni il valore corrispondente nella colonna 'DEV URLS'
        dev_url = new_urls.at[best_match_index, 'DEV URLS']
        return best_match, best_score, dev_url
    else:
        return None, 0, None

print("Finding FuzzyWuzzy best matches...")

def apply_find_best_match(x):
    return find_best_match(x, new_urls)

# Creare una copia di 'old_urls'
old_urls_copy = old_urls.copy()
total_urls = len(old_urls_copy['Cleaned LIVE URLS'])

for i, url in enumerate(old_urls_copy['Cleaned LIVE URLS']):
    old_urls_copy.loc[i, 'Cleaned DEV URLS'], old_urls_copy.loc[i, 'SIMILARITY'], old_urls_copy.loc[i, 'DEV URLS'] = apply_find_best_match(url)
    if i % 10 == 0:  # update every xx iterations
        print(f"\rProcessed {i} out of {total_urls} URLs.", end="")
#print("Processing complete.")
print("\nProcessing complete.")
matched_urls = old_urls_copy
print(f"Fuzzy matched {len(matched_urls)} out of {len(old_urls_copy)} URLs.")

redirect_map_fuzzywuzzy = matched_urls[['LIVE URLS', 'DEV URLS', 'Cleaned LIVE URLS', 'Cleaned DEV URLS', 'SIMILARITY']]
redirect_map_fuzzywuzzy = redirect_map_fuzzywuzzy.rename(columns={"SIMILARITY": "Fuzzy Similiarity"})


Finding FuzzyWuzzy best matches...


Processed 40 out of 49 URLs.
Processing complete.
Fuzzy matched 49 out of 49 URLs.


In [57]:
matched_urls.head(5)

,LIVE URLS,Cleaned LIVE URLS,Cleaned DEV URLS,SIMILARITY,DEV URLS
0,https://www.nonnapaperina.it/2014/04/la-federc...,2014 04 la federcasalinghe e il mondo delle i...,2019 08 purobio home per contrastare intoller...,50.0,https://www.nonnapaperina.it/2019/08/purobio-h...
1,https://www.nonnapaperina.it/2014/04/limportan...,2014 04 limportanza delle pentole per la nost...,2020 05 fiorellini di polenta con salmone per...,57.0,https://www.nonnapaperina.it/2020/05/fiorellin...
2,https://www.nonnapaperina.it/2017/10/petto-dan...,2017 10 petto danatra affumicato con foglie d...,2017 10 petto di anatra affumicato con foglie...,83.0,https://www.nonnapaperina.it/2017/10/petto-di-...
3,https://www.nonnapaperina.it/2013/03/fave-con-...,2013 03 fave con pancetta,2016 07 pasta con feta,72.0,https://www.nonnapaperina.it/2016/07/pasta-con...
4,https://www.nonnapaperina.it/2016/05/celiachia...,2016 05 celiachia un protocollo migliorare la...,2021 09 millefoglie di tortillas con pollo e ...,57.0,https://www.nonnapaperina.it/2021/09/millefogl...


In [58]:
redirect_map_fuzzywuzzy.head(5)

,LIVE URLS,DEV URLS,Cleaned LIVE URLS,Cleaned DEV URLS,Fuzzy Similiarity
0,https://www.nonnapaperina.it/2014/04/la-federc...,https://www.nonnapaperina.it/2019/08/purobio-h...,2014 04 la federcasalinghe e il mondo delle i...,2019 08 purobio home per contrastare intoller...,50.0
1,https://www.nonnapaperina.it/2014/04/limportan...,https://www.nonnapaperina.it/2020/05/fiorellin...,2014 04 limportanza delle pentole per la nost...,2020 05 fiorellini di polenta con salmone per...,57.0
2,https://www.nonnapaperina.it/2017/10/petto-dan...,https://www.nonnapaperina.it/2017/10/petto-di-...,2017 10 petto danatra affumicato con foglie d...,2017 10 petto di anatra affumicato con foglie...,83.0
3,https://www.nonnapaperina.it/2013/03/fave-con-...,https://www.nonnapaperina.it/2016/07/pasta-con...,2013 03 fave con pancetta,2016 07 pasta con feta,72.0
4,https://www.nonnapaperina.it/2016/05/celiachia...,https://www.nonnapaperina.it/2021/09/millefogl...,2016 05 celiachia un protocollo migliorare la...,2021 09 millefoglie di tortillas con pollo e ...,57.0


In [59]:
old_urls.head(5)

,LIVE URLS,Cleaned LIVE URLS
0,https://www.nonnapaperina.it/2014/04/la-federc...,2014 04 la federcasalinghe e il mondo delle i...
1,https://www.nonnapaperina.it/2014/04/limportan...,2014 04 limportanza delle pentole per la nost...
2,https://www.nonnapaperina.it/2017/10/petto-dan...,2017 10 petto danatra affumicato con foglie d...
3,https://www.nonnapaperina.it/2013/03/fave-con-...,2013 03 fave con pancetta
4,https://www.nonnapaperina.it/2016/05/celiachia...,2016 05 celiachia un protocollo migliorare la...


In [60]:
new_urls.head(5)

,DEV URLS,Cleaned DEV URLS
0,https://www.nonnapaperina.it/,
1,https://www.nonnapaperina.it/pillole/,pillole
2,https://www.nonnapaperina.it/ingredienti/,ingredienti
3,https://www.nonnapaperina.it/scuola-di-cucina/...,scuola di cucina impariamo ad essiccare
4,https://www.nonnapaperina.it/contattami/,contattami


In [61]:
# fine fuzzy
# inizio difflib

In [62]:
# utilizza il modulo difflib per cercare corrispondenze strette tra gli URL di riferimento
import difflib

# Create a new dataframe to store the results
url_mapping_list = []

#counter
progress_interval = 10  # Intervallo di progressione
counter = 0  # Contatore per tenere traccia del progresso
#fine counter

for old_url in old_urls["Cleaned LIVE URLS"]:
    matches = difflib.get_close_matches(old_url, new_urls["Cleaned DEV URLS"])
    if matches:
        url_mapping_list.append({"LIVE URLS": old_urls.loc[old_urls["Cleaned LIVE URLS"] == old_url, "LIVE URLS"].iloc[0],
                                 "Cleaned LIVE URLS": old_url,
                                 "DEV URLS": new_urls.loc[new_urls["Cleaned DEV URLS"] == matches[0], "DEV URLS"].iloc[0],
                                 "Cleaned DEV URLS": matches[0]})
        #print('match found')
    else:
        url_mapping_list.append({"LIVE URLS": old_urls.loc[old_urls["Cleaned LIVE URLS"] == old_url, "LIVE URLS"].iloc[0],
                                 "Cleaned LIVE URLS": old_url,
                                 "DEV URLS": None,
                                 "Cleaned DEV URLS": None})
        #print('match not found')
    # Aggiungi una riga di stampa per il progresso ogni intervallo specificato
    counter += 1
    if counter % progress_interval == 0:
        #print(f"Progresso DiffLib: {counter}/{len(old_urls['Cleaned LIVE URLS'])}")
        print(f"\rProgresso DiffLib: {counter}/{len(old_urls['Cleaned LIVE URLS'])}", end='', flush=True)

# Convert the list to a DataFrame
#url_mapping = pd.DataFrame(url_mapping_list)
redirect_map_difflib = pd.DataFrame(url_mapping_list)
# cambia ordine colonne
redirect_map_difflib = redirect_map_difflib[['LIVE URLS', 'DEV URLS', 'Cleaned LIVE URLS', 'Cleaned DEV URLS']]

redirect_map_difflib.head(5)

Progresso DiffLib: 40/49

,LIVE URLS,DEV URLS,Cleaned LIVE URLS,Cleaned DEV URLS
0,https://www.nonnapaperina.it/2014/04/la-federc...,None,2014 04 la federcasalinghe e il mondo delle i...,None
1,https://www.nonnapaperina.it/2014/04/limportan...,https://www.nonnapaperina.it/2017/01/limportan...,2014 04 limportanza delle pentole per la nost...,2017 01 limportanza della prima colazione per...
2,https://www.nonnapaperina.it/2017/10/petto-dan...,https://www.nonnapaperina.it/2017/10/petto-di-...,2017 10 petto danatra affumicato con foglie d...,2017 10 petto di anatra affumicato con foglie...
3,https://www.nonnapaperina.it/2013/03/fave-con-...,https://www.nonnapaperina.it/2016/07/pasta-con...,2013 03 fave con pancetta,2016 07 pasta con feta
4,https://www.nonnapaperina.it/2016/05/celiachia...,https://www.nonnapaperina.it/2018/04/celiachia...,2016 05 celiachia un protocollo migliorare la...,2018 04 celiachia e malattie rare difficili d...


In [63]:
# Create a new list to store the mapped URLs
mapped_urls2 = []

# counter
counter = 0  # Contatore per tenere traccia del progresso
# fine counter

for old_url, cleaned_live_url in zip(old_urls["LIVE URLS"], old_urls["Cleaned LIVE URLS"]):
    max_ratio = 0
    best_new_url = None
    best_cleaned_dev_url = None
    best_similarity = 0  # Inizializza la similarità migliore a 0
    for new_url, cleaned_dev_url in zip(new_urls["DEV URLS"], new_urls["Cleaned DEV URLS"]):
        match = difflib.SequenceMatcher(None, cleaned_live_url, cleaned_dev_url)
        ratio = match.ratio()
        if ratio > max_ratio:
            max_ratio = ratio
            best_new_url = new_url
            best_cleaned_dev_url = cleaned_dev_url
            best_similarity = ratio  # Aggiorna la similarità migliore
    if best_similarity > 0:
        mapped_urls2.append({"LIVE URLS": old_url, "Cleaned LIVE URLS": cleaned_live_url, "DEV URLS": best_new_url, "Cleaned DEV URLS": best_cleaned_dev_url, "Similarity": best_similarity})
    else:
        mapped_urls2.append({"LIVE URLS": old_url, "Cleaned LIVE URLS": cleaned_live_url, "DEV URLS": None, "Cleaned DEV URLS": None, "Similarity": best_similarity})

    # Aggiungi una riga di stampa per il progresso ogni intervallo specificato
    counter += 1
    if counter % progress_interval == 0:
        #print(f"Progresso: {counter}/{len(old_urls['LIVE URLS'])}")
        print(f"\rProgresso Difflib SequenceMatcher: {counter}/{len(old_urls['LIVE URLS'])}", end='', flush=True)

# Convert the list to a DataFrame
redirect_map_difflib2 = pd.DataFrame(mapped_urls2)

# cambia ordine colonne
redirect_map_difflib2 = redirect_map_difflib2[['LIVE URLS', 'DEV URLS', 'Cleaned LIVE URLS', 'Cleaned DEV URLS','Similarity']]
redirect_map_difflib2 = redirect_map_difflib2.rename(columns={"Similarity": "Difflib SM Similarity"})

Progresso Difflib SequenceMatcher: 40/49

In [64]:
redirect_map_difflib2.head(5)

,LIVE URLS,DEV URLS,Cleaned LIVE URLS,Cleaned DEV URLS,Difflib SM Similarity
0,https://www.nonnapaperina.it/2014/04/la-federc...,https://www.nonnapaperina.it/2016/08/breath-te...,2014 04 la federcasalinghe e il mondo delle i...,2016 08 breath test esame intolleranza al lat...,0.483660
1,https://www.nonnapaperina.it/2014/04/limportan...,https://www.nonnapaperina.it/2017/01/limportan...,2014 04 limportanza delle pentole per la nost...,2017 01 limportanza della prima colazione per...,0.628099
2,https://www.nonnapaperina.it/2017/10/petto-dan...,https://www.nonnapaperina.it/2017/10/petto-di-...,2017 10 petto danatra affumicato con foglie d...,2017 10 petto di anatra affumicato con foglie...,0.938053
3,https://www.nonnapaperina.it/2013/03/fave-con-...,https://www.nonnapaperina.it/2016/07/pasta-con...,2013 03 fave con pancetta,2016 07 pasta con feta,0.666667
4,https://www.nonnapaperina.it/2016/05/celiachia...,https://www.nonnapaperina.it/2018/04/celiachia...,2016 05 celiachia un protocollo migliorare la...,2018 04 celiachia e malattie rare difficili d...,0.619048


In [65]:
# unione DF
###########

# Unire old_urls e redirect_map_fuzzywuzzy
merged_df = old_urls.merge(redirect_map_fuzzywuzzy, on='LIVE URLS', how='left')
# cancella colonne extra
merged_df = merged_df.drop(['Cleaned LIVE URLS_x', 'Cleaned LIVE URLS_y','Cleaned DEV URLS'], axis=1)
merged_df.head(5)

,LIVE URLS,DEV URLS,Fuzzy Similiarity
0,https://www.nonnapaperina.it/2014/04/la-federc...,https://www.nonnapaperina.it/2019/08/purobio-h...,50.0
1,https://www.nonnapaperina.it/2014/04/limportan...,https://www.nonnapaperina.it/2020/05/fiorellin...,57.0
2,https://www.nonnapaperina.it/2017/10/petto-dan...,https://www.nonnapaperina.it/2017/10/petto-di-...,83.0
3,https://www.nonnapaperina.it/2013/03/fave-con-...,https://www.nonnapaperina.it/2016/07/pasta-con...,72.0
4,https://www.nonnapaperina.it/2016/05/celiachia...,https://www.nonnapaperina.it/2021/09/millefogl...,57.0


In [66]:
# Unire url_mapping a difflib
merged_df = merged_df.merge(redirect_map_difflib, left_on='LIVE URLS', right_on='LIVE URLS', how='left')
# cancella colonne extra
merged_df = merged_df.drop(['Cleaned LIVE URLS','Cleaned DEV URLS'], axis=1)
# rinomina colonne
merged_df = merged_df.rename(columns={"DEV URLS_x": "Fuzzy", "DEV URLS_y": "Difflib"})


merged_df.head(5)

,LIVE URLS,Fuzzy,Fuzzy Similiarity,Difflib
0,https://www.nonnapaperina.it/2014/04/la-federc...,https://www.nonnapaperina.it/2019/08/purobio-h...,50.0,None
1,https://www.nonnapaperina.it/2014/04/limportan...,https://www.nonnapaperina.it/2020/05/fiorellin...,57.0,https://www.nonnapaperina.it/2017/01/limportan...
2,https://www.nonnapaperina.it/2017/10/petto-dan...,https://www.nonnapaperina.it/2017/10/petto-di-...,83.0,https://www.nonnapaperina.it/2017/10/petto-di-...
3,https://www.nonnapaperina.it/2013/03/fave-con-...,https://www.nonnapaperina.it/2016/07/pasta-con...,72.0,https://www.nonnapaperina.it/2016/07/pasta-con...
4,https://www.nonnapaperina.it/2016/05/celiachia...,https://www.nonnapaperina.it/2021/09/millefogl...,57.0,https://www.nonnapaperina.it/2018/04/celiachia...


In [67]:
# Unire mapped_df a merged_df
merged_df = merged_df.merge(redirect_map_difflib2, on='LIVE URLS', how='left')
#merged_df = merged_df.merge(mapped_df[['LIVE URLS', 'Similarity']], on='LIVE URLS', how='left')
# cancella colonne extra
merged_df = merged_df.drop(['Cleaned LIVE URLS','Cleaned DEV URLS'], axis=1)
merged_df = merged_df.rename(columns={"DEV URLS": "Difflib SequenceMatcher"})
merged_df.head(5)

,LIVE URLS,Fuzzy,Fuzzy Similiarity,Difflib,Difflib SequenceMatcher,Difflib SM Similarity
0,https://www.nonnapaperina.it/2014/04/la-federc...,https://www.nonnapaperina.it/2019/08/purobio-h...,50.0,None,https://www.nonnapaperina.it/2016/08/breath-te...,0.483660
1,https://www.nonnapaperina.it/2014/04/limportan...,https://www.nonnapaperina.it/2020/05/fiorellin...,57.0,https://www.nonnapaperina.it/2017/01/limportan...,https://www.nonnapaperina.it/2017/01/limportan...,0.628099
2,https://www.nonnapaperina.it/2017/10/petto-dan...,https://www.nonnapaperina.it/2017/10/petto-di-...,83.0,https://www.nonnapaperina.it/2017/10/petto-di-...,https://www.nonnapaperina.it/2017/10/petto-di-...,0.938053
3,https://www.nonnapaperina.it/2013/03/fave-con-...,https://www.nonnapaperina.it/2016/07/pasta-con...,72.0,https://www.nonnapaperina.it/2016/07/pasta-con...,https://www.nonnapaperina.it/2016/07/pasta-con...,0.666667
4,https://www.nonnapaperina.it/2016/05/celiachia...,https://www.nonnapaperina.it/2021/09/millefogl...,57.0,https://www.nonnapaperina.it/2018/04/celiachia...,https://www.nonnapaperina.it/2018/04/celiachia...,0.619048


In [68]:
# similitudine con spacy
import spacy

counter = 0  # Contatore per tenere traccia del progresso
progress_interval = 1  # Intervallo di progressione

# Carica il modello di linguaggio italiano
nlp = spacy.load("it_core_news_lg")
# Carica il modello di linguaggio inglese
#nlp = spacy.load("en_core_news_lg")

# Crea una lista di documenti spaCy per old_urls e new_urls
old_docs = [nlp(url) for url in old_urls['Cleaned LIVE URLS']]
new_docs = [nlp(url) for url in new_urls['Cleaned DEV URLS']]

# Trova l'URL più simile in new_urls per ogni URL in old_urls
most_similar_urls = []

for old_doc in old_docs:
    similarities = [old_doc.similarity(new_doc) for new_doc in new_docs]
    most_similar_index = similarities.index(max(similarities))
    most_similar_urls.append(new_urls['DEV URLS'].iloc[most_similar_index])
    counter += 1  # Contatore per tenere traccia del progresso
    if counter % progress_interval == 0:
        print(f"\rProgresso Spacy: {counter}/{len(old_urls['Cleaned LIVE URLS'])}", end='', flush=True)

# Crea un DataFrame con i risultati
spacy_df = pd.DataFrame({
    'LIVE URLS': old_urls['LIVE URLS'],
    'Spacy': most_similar_urls
})

spacy_df.head(5)


/tmp/ipykernel_4062/3300374611.py:20: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarities = [old_doc.similarity(new_doc) for new_doc in new_docs]


Progresso Spacy: 49/49

,LIVE URLS,Spacy
0,https://www.nonnapaperina.it/2014/04/la-federc...,https://www.nonnapaperina.it/2021/05/il-beness...
1,https://www.nonnapaperina.it/2014/04/limportan...,https://www.nonnapaperina.it/2016/09/potassio-...
2,https://www.nonnapaperina.it/2017/10/petto-dan...,https://www.nonnapaperina.it/2017/10/petto-di-...
3,https://www.nonnapaperina.it/2013/03/fave-con-...,https://www.nonnapaperina.it/2013/03/risotto-c...
4,https://www.nonnapaperina.it/2016/05/celiachia...,https://www.nonnapaperina.it/2016/07/coste-gra...


In [69]:
# Unire spacy_df a merged_df
merged_df = merged_df.merge(spacy_df, on='LIVE URLS', how='left')
merged_df.head(5)

,LIVE URLS,Fuzzy,Fuzzy Similiarity,Difflib,Difflib SequenceMatcher,Difflib SM Similarity,Spacy
0,https://www.nonnapaperina.it/2014/04/la-federc...,https://www.nonnapaperina.it/2019/08/purobio-h...,50.0,None,https://www.nonnapaperina.it/2016/08/breath-te...,0.483660,https://www.nonnapaperina.it/2021/05/il-beness...
1,https://www.nonnapaperina.it/2014/04/limportan...,https://www.nonnapaperina.it/2020/05/fiorellin...,57.0,https://www.nonnapaperina.it/2017/01/limportan...,https://www.nonnapaperina.it/2017/01/limportan...,0.628099,https://www.nonnapaperina.it/2016/09/potassio-...
2,https://www.nonnapaperina.it/2017/10/petto-dan...,https://www.nonnapaperina.it/2017/10/petto-di-...,83.0,https://www.nonnapaperina.it/2017/10/petto-di-...,https://www.nonnapaperina.it/2017/10/petto-di-...,0.938053,https://www.nonnapaperina.it/2017/10/petto-di-...
3,https://www.nonnapaperina.it/2013/03/fave-con-...,https://www.nonnapaperina.it/2016/07/pasta-con...,72.0,https://www.nonnapaperina.it/2016/07/pasta-con...,https://www.nonnapaperina.it/2016/07/pasta-con...,0.666667,https://www.nonnapaperina.it/2013/03/risotto-c...
4,https://www.nonnapaperina.it/2016/05/celiachia...,https://www.nonnapaperina.it/2021/09/millefogl...,57.0,https://www.nonnapaperina.it/2018/04/celiachia...,https://www.nonnapaperina.it/2018/04/celiachia...,0.619048,https://www.nonnapaperina.it/2016/07/coste-gra...


In [73]:
# similitudine con ngram
import pandas as pd
from ngram import NGram

# Supponiamo che old_urls e new_urls siano i tuoi DataFrame

# Inizializza NGram
n = NGram()

counter = 0  # Contatore per tenere traccia del progresso
progress_interval = 100  # Intervallo di progressione

# Funzione per trovare l'URL più simile
def find_most_similar_url(old_url, new_urls_list):
    global counter
    max_similarity = -1
    most_similar_url = None
    for new_url in new_urls_list:
        similarity = n.compare(old_url, new_url)

        counter += 1  # Contatore per tenere traccia del progresso
        if counter % progress_interval == 0:
            print(f"\rProgresso Ngram: {counter}/{len(old_urls['Cleaned LIVE URLS'])}", end='', flush=True)

        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_url = new_url
    return most_similar_url

# Calcola l'URL più simile per ogni URL in old_urls
most_similar_urls = [find_most_similar_url(old_url, new_urls['DEV URLS'].tolist()) for old_url in old_urls['LIVE URLS']]

# Crea un DataFrame con i risultati
ngram_df = pd.DataFrame({
    'LIVE URLS': old_urls['LIVE URLS'],
    'Ngram': most_similar_urls
})

ngram_df.head(5)


Progresso Ngram: 283024/49

,LIVE URLS,Ngram
0,https://www.nonnapaperina.it/2014/04/la-federc...,https://www.nonnapaperina.it/2015/01/tisane-in...
1,https://www.nonnapaperina.it/2014/04/limportan...,https://www.nonnapaperina.it/2015/05/farina-av...
2,https://www.nonnapaperina.it/2017/10/petto-dan...,https://www.nonnapaperina.it/2017/10/petto-di-...
3,https://www.nonnapaperina.it/2013/03/fave-con-...,https://www.nonnapaperina.it/2015/03/risotto-c...
4,https://www.nonnapaperina.it/2016/05/celiachia...,https://www.nonnapaperina.it/2017/01/la-diagno...


In [ ]:
# Unire ngram_df a merged_df
merged_df = merged_df.merge(ngram_df, on='LIVE URLS', how='left')
merged_df.head(5)

,LIVE URLS,Fuzzy,Fuzzy Similiarity,Difflib,Difflib SequenceMatcher,Difflib SM Similarity,Spacy,Ngram
0,https://www.nonnapaperina.it/2014/04/la-federc...,https://www.nonnapaperina.it/2019/08/purobio-h...,50.0,None,https://www.nonnapaperina.it/2016/08/breath-te...,0.483660,https://www.nonnapaperina.it/2021/05/il-beness...,https://www.nonnapaperina.it/2015/01/tisane-in...
1,https://www.nonnapaperina.it/2014/04/limportan...,https://www.nonnapaperina.it/2020/05/fiorellin...,57.0,https://www.nonnapaperina.it/2017/01/limportan...,https://www.nonnapaperina.it/2017/01/limportan...,0.628099,https://www.nonnapaperina.it/2016/09/potassio-...,https://www.nonnapaperina.it/2015/05/farina-av...
2,https://www.nonnapaperina.it/2017/10/petto-dan...,https://www.nonnapaperina.it/2017/10/petto-di-...,83.0,https://www.nonnapaperina.it/2017/10/petto-di-...,https://www.nonnapaperina.it/2017/10/petto-di-...,0.938053,https://www.nonnapaperina.it/2017/10/petto-di-...,https://www.nonnapaperina.it/2017/10/petto-di-...
3,https://www.nonnapaperina.it/2013/03/fave-con-...,https://www.nonnapaperina.it/2016/07/pasta-con...,72.0,https://www.nonnapaperina.it/2016/07/pasta-con...,https://www.nonnapaperina.it/2016/07/pasta-con...,0.666667,https://www.nonnapaperina.it/2013/03/risotto-c...,https://www.nonnapaperina.it/2015/03/risotto-c...
4,https://www.nonnapaperina.it/2016/05/celiachia...,https://www.nonnapaperina.it/2021/09/millefogl...,57.0,https://www.nonnapaperina.it/2018/04/celiachia...,https://www.nonnapaperina.it/2018/04/celiachia...,0.619048,https://www.nonnapaperina.it/2016/07/coste-gra...,https://www.nonnapaperina.it/2017/01/la-diagno...


In [ ]:
# Rinominare le colonne
merged_df = merged_df.rename(columns={"LIVE URLS":"URL da redirezionare"})

# modifica valore similarity arrotondando *100
merged_df['Difflib SM Similarity'] = (merged_df['Difflib SM Similarity'] * 100).round()

# Stampare il dataframe finale
merged_df.head(5)

,URL da redirezionare,Fuzzy,Fuzzy Similiarity,Difflib,Difflib SequenceMatcher,Difflib SM Similarity,Spacy,Ngram
0,https://www.nonnapaperina.it/2014/04/la-federc...,https://www.nonnapaperina.it/2019/08/purobio-h...,50.0,None,https://www.nonnapaperina.it/2016/08/breath-te...,48.0,https://www.nonnapaperina.it/2021/05/il-beness...,https://www.nonnapaperina.it/2015/01/tisane-in...
1,https://www.nonnapaperina.it/2014/04/limportan...,https://www.nonnapaperina.it/2020/05/fiorellin...,57.0,https://www.nonnapaperina.it/2017/01/limportan...,https://www.nonnapaperina.it/2017/01/limportan...,63.0,https://www.nonnapaperina.it/2016/09/potassio-...,https://www.nonnapaperina.it/2015/05/farina-av...
2,https://www.nonnapaperina.it/2017/10/petto-dan...,https://www.nonnapaperina.it/2017/10/petto-di-...,83.0,https://www.nonnapaperina.it/2017/10/petto-di-...,https://www.nonnapaperina.it/2017/10/petto-di-...,94.0,https://www.nonnapaperina.it/2017/10/petto-di-...,https://www.nonnapaperina.it/2017/10/petto-di-...
3,https://www.nonnapaperina.it/2013/03/fave-con-...,https://www.nonnapaperina.it/2016/07/pasta-con...,72.0,https://www.nonnapaperina.it/2016/07/pasta-con...,https://www.nonnapaperina.it/2016/07/pasta-con...,67.0,https://www.nonnapaperina.it/2013/03/risotto-c...,https://www.nonnapaperina.it/2015/03/risotto-c...
4,https://www.nonnapaperina.it/2016/05/celiachia...,https://www.nonnapaperina.it/2021/09/millefogl...,57.0,https://www.nonnapaperina.it/2018/04/celiachia...,https://www.nonnapaperina.it/2018/04/celiachia...,62.0,https://www.nonnapaperina.it/2016/07/coste-gra...,https://www.nonnapaperina.it/2017/01/la-diagno...


In [ ]:
def count_identical_values(row):
    # Estrai i valori dalle colonne specificate
    values = [row['Fuzzy'], row['Difflib'], row['Difflib SequenceMatcher'], row['Spacy']]
    
    # Rimuovi valori None o vuoti dalla lista
    non_none_values = [v for v in values if v is not None and v != '']
    
    # Calcola il numero di elementi unici
    unique_count = len(set(non_none_values))
    
    # Restituisci il conteggio degli elementi identici
    return len(non_none_values) - unique_count + 1

merged_df['Agreements'] = merged_df.apply(count_identical_values, axis=1)
merged_df


,URL da redirezionare,Fuzzy,Fuzzy Similiarity,Difflib,Difflib SequenceMatcher,Difflib SM Similarity,Spacy,Ngram,Agreements
0,https://www.nonnapaperina.it/2014/04/la-federc...,https://www.nonnapaperina.it/2019/08/purobio-h...,50.0,None,https://www.nonnapaperina.it/2016/08/breath-te...,48.0,https://www.nonnapaperina.it/2021/05/il-beness...,https://www.nonnapaperina.it/2015/01/tisane-in...,1
1,https://www.nonnapaperina.it/2014/04/limportan...,https://www.nonnapaperina.it/2020/05/fiorellin...,57.0,https://www.nonnapaperina.it/2017/01/limportan...,https://www.nonnapaperina.it/2017/01/limportan...,63.0,https://www.nonnapaperina.it/2016/09/potassio-...,https://www.nonnapaperina.it/2015/05/farina-av...,2
2,https://www.nonnapaperina.it/2017/10/petto-dan...,https://www.nonnapaperina.it/2017/10/petto-di-...,83.0,https://www.nonnapaperina.it/2017/10/petto-di-...,https://www.nonnapaperina.it/2017/10/petto-di-...,94.0,https://www.nonnapaperina.it/2017/10/petto-di-...,https://www.nonnapaperina.it/2017/10/petto-di-...,4
3,https://www.nonnapaperina.it/2013/03/fave-con-...,https://www.nonnapaperina.it/2016/07/pasta-con...,72.0,https://www.nonnapaperina.it/2016/07/pasta-con...,https://www.nonnapaperina.it/2016/07/pasta-con...,67.0,https://www.nonnapaperina.it/2013/03/risotto-c...,https://www.nonnapaperina.it/2015/03/risotto-c...,3
4,https://www.nonnapaperina.it/2016/05/celiachia...,https://www.nonnapaperina.it/2021/09/millefogl...,57.0,https://www.nonnapaperina.it/2018/04/celiachia...,https://www.nonnapaperina.it/2018/04/celiachia...,62.0,https://www.nonnapaperina.it/2016/07/coste-gra...,https://www.nonnapaperina.it/2017/01/la-diagno...,2
5,https://www.nonnapaperina.it/2015/08/pasta-fre...,https://www.nonnapaperina.it/2017/10/pasta-fro...,77.0,https://www.nonnapaperina.it/2017/10/pasta-fro...,https://www.nonnapaperina.it/2017/10/pasta-fro...,78.0,https://www.nonnapaperina.it/2017/04/mousse-mo...,https://www.nonnapaperina.it/2015/06/farina-di...,3
6,https://www.nonnapaperina.it/2017/03/brianza-c...,https://www.nonnapaperina.it/2018/04/branzino-...,58.0,None,https://www.nonnapaperina.it/2018/09/ravioli-t...,58.0,https://www.nonnapaperina.it/2015/07/passione-...,https://www.nonnapaperina.it/2017/01/karela/,1
7,https://www.nonnapaperina.it/2011/09/chiodini-...,https://www.nonnapaperina.it/2019/02/penne-con...,63.0,https://www.nonnapaperina.it/2016/08/spiedini-...,https://www.nonnapaperina.it/2016/08/spiedini-...,64.0,https://www.nonnapaperina.it/2016/09/fusilli-c...,https://www.nonnapaperina.it/2013/11/prezzemolo/,2
8,https://www.nonnapaperina.it/2012/08/grissini/...,https://www.nonnapaperina.it/2012/08/grissini-...,71.0,https://www.nonnapaperina.it/2012/08/grissini-...,https://www.nonnapaperina.it/2012/08/grissini-...,73.0,https://www.nonnapaperina.it/2014/08/birramisu/,https://www.nonnapaperina.it/2012/08/grissini-...,3
9,https://www.nonnapaperina.it/2016/06/tenuta-di...,https://www.nonnapaperina.it/2016/10/farina-di...,63.0,https://www.nonnapaperina.it/2016/04/insalata-...,https://www.nonnapaperina.it/2016/04/insalata-...,67.0,https://www.nonnapaperina.it/2016/06/condiment...,https://www.nonnapaperina.it/2016/06/cappuccino/,2


In [ ]:
# prepara ambiente
output_folder_path = "./output"
current_date_time = datetime.datetime.now().strftime("%Y%m%d_%H%M")
output_filename = f"{current_date_time}_redirect_map.xlsx"
output_path = os.path.join(output_folder_path, output_filename)

with pd.ExcelWriter(output_path) as writer:
    print("Saving redirect maps to Excel...")
    #redirect_map_fuzzywuzzy.to_excel(writer, sheet_name='FuzzyWuzzy', index=False)
    #redirect_map_difflib.to_excel(writer, sheet_name='Difflib', index=False)
    #mapped_df.to_excel(writer, sheet_name='SequenceMatcher', index=False)
    merged_df.to_excel(writer, sheet_name='Mapping', index=False)
    print(f"Redirect maps creati con successo! Nome del file: {output_filename}")

Saving redirect maps to Excel...
Redirect maps creati con successo! Nome del file: 20230802_2305_redirect_map.xlsx
